# Resampling and Frequency Conversion（重采样和频度转换）

重采样（Resampling）指的是把时间序列的频度变为另一个频度的过程。把高频度的数据变为低频度叫做降采样（downsampling），把低频度变为高频度叫做增采样（upsampling）。并不是所有的重采样都会落入上面这几个类型，例如，把W-WED（weekly on Wednesday）变为W-FRI，既不属于降采样，也不属于增采样。

pandas对象自带resampe方法，用于所有的频度变化。resample有一个和groupby类似的API；我们可以用resample来对数据进行分组，然后调用聚合函数（aggregation function）:

In [1]:
import numpy as np
import pandas as pd

In [2]:
rng = pd.date_range('2000-01-01', periods=100, freq='D')

In [3]:
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

2000-01-01   -1.400167
2000-01-02    1.244175
2000-01-03    1.321916
2000-01-04    1.311636
2000-01-05   -1.039417
                ...   
2000-04-05    0.183185
2000-04-06    0.301526
2000-04-07    0.594882
2000-04-08   -0.172711
2000-04-09   -0.031627
Freq: D, Length: 100, dtype: float64

In [4]:
ts.resample('M').mean()

2000-01-31    0.098941
2000-02-29   -0.367203
2000-03-31   -0.076423
2000-04-30    0.094108
Freq: M, dtype: float64



# 1 Downsampling（降采样）

把数据聚合为规律、低频度是一个很普通的时间序列任务。用于处理的数据不必是有固定频度的；我们想要设定的频度会定义箱界（bin edges），根据bin edges会把时间序列分割为多个片段，然后进行聚合。例如，转换为月度，比如'M'或'BM'，我们需要把数据以月为间隔进行切割。每一个间隔都是半开放的（half-open）；一个数据点只能属于一个间隔，所有间隔的合集，构成整个时间范围（time frame）。当使用resample去降采样数据的时候，有很多事情需要考虑：

- 在每个间隔里，哪一边要闭合
- 怎样对每一个聚合的bin贴标签，可以使用间隔的开始或结束

为了演示一下，下面用一个一分钟的数据来举例：

In [5]:
rng = pd.date_range('2000-01-01', periods=12, freq='T')

In [6]:
ts = pd.Series(np.arange(12), index=rng)
ts

2000-01-01 00:00:00     0
2000-01-01 00:01:00     1
2000-01-01 00:02:00     2
2000-01-01 00:03:00     3
2000-01-01 00:04:00     4
2000-01-01 00:05:00     5
2000-01-01 00:06:00     6
2000-01-01 00:07:00     7
2000-01-01 00:08:00     8
2000-01-01 00:09:00     9
2000-01-01 00:10:00    10
2000-01-01 00:11:00    11
Freq: T, dtype: int64

假设我们想要按5分钟一个数据块来进行聚合，然后对每一个组计算总和：

In [11]:
ts.resample('5min', closed='left').sum()

2000-01-01 00:00:00    10
2000-01-01 00:05:00    35
2000-01-01 00:10:00    21
Freq: 5T, dtype: int64

In [10]:
ts.resample('5min', closed='right').sum()

1999-12-31 23:55:00     0
2000-01-01 00:00:00    15
2000-01-01 00:05:00    40
2000-01-01 00:10:00    11
Freq: 5T, dtype: int64

默认，每一个bin的左边的时间戳，会被用来作为结果里时间序列的标签。通过设置label='right'，我们可以使用bin右边的时间戳来作为标签：

In [17]:
ts.resample('5min', closed='right', label='right').sum()

2000-01-01 00:00:00     0
2000-01-01 00:05:00    15
2000-01-01 00:10:00    40
2000-01-01 00:15:00    11
Freq: 5T, dtype: int64



最后，我们可能想要对结果的索引进行位移，比如在右边界减少一秒。想要实现的话，传递一个字符串或日期偏移给loffset：

In [12]:
ts.resample('5min', closed='right', 
            label='right', loffset='-1s').sum()

1999-12-31 23:59:59     0
2000-01-01 00:04:59    15
2000-01-01 00:09:59    40
2000-01-01 00:14:59    11
Freq: 5T, dtype: int64

我们也可以使用shift方法来实现上面loffset的效果。

### Open-High-Low-Close (OHLC) resampling（股价图重取样）

>  Open-High-Low-Close: 开盘-盘高-盘低-收盘图；股票图；股价图

在经济界，一个比较流行的用法，是对时间序列进行聚合，计算每一个桶（bucket）里的四个值：first（open），last（close），maximum（high），minimal（low），即开盘-收盘-盘高-盘低，四个值。使用ohlc聚合函数可以得到这四个聚合结果：

In [13]:
ts.resample('5min').ohlc()

,open,high,low,close
2000-01-01 00:00:00,0,4,0,4
2000-01-01 00:05:00,5,9,5,9
2000-01-01 00:10:00,10,11,10,11


# 2 Upsampling and Interpolation（增采样和插值）

把一个低频度转换为高频度，是不需要进行聚合的。下面是一个有周数据的DataFrame：

In [20]:
frame = pd.DataFrame(np.random.randn(2, 4),
                     index=pd.date_range('1/1/2000', periods=2,
                                         freq='W-WED'),
                     columns=['Colorado', 'Texas', 'New York', 'Ohio'])
frame

,Colorado,Texas,New York,Ohio
2000-01-05,0.138355,1.881517,0.655367,1.496932
2000-01-12,-1.125212,-0.824337,0.803721,-0.672660


当我们对这个数据进行聚合的的时候，每个组只有一个值，以及gap（间隔）之间的缺失值。在不使用任何聚合函数的情况下，我们使用asfreq方法将其转换为高频度：

In [21]:
df_daily = frame.resample('D').asfreq()
df_daily

,Colorado,Texas,New York,Ohio
2000-01-05,0.138355,1.881517,0.655367,1.496932
2000-01-06,NaN,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN,NaN
2000-01-08,NaN,NaN,NaN,NaN
2000-01-09,NaN,NaN,NaN,NaN
2000-01-10,NaN,NaN,NaN,NaN
2000-01-11,NaN,NaN,NaN,NaN
2000-01-12,-1.125212,-0.824337,0.803721,-0.672660


In [22]:
frame.resample('D').ffill()

,Colorado,Texas,New York,Ohio
2000-01-05,0.138355,1.881517,0.655367,1.496932
2000-01-06,0.138355,1.881517,0.655367,1.496932
2000-01-07,0.138355,1.881517,0.655367,1.496932
2000-01-08,0.138355,1.881517,0.655367,1.496932
2000-01-09,0.138355,1.881517,0.655367,1.496932
2000-01-10,0.138355,1.881517,0.655367,1.496932
2000-01-11,0.138355,1.881517,0.655367,1.496932
2000-01-12,-1.125212,-0.824337,0.803721,-0.672660


我们可以选择只对一部分的周期进行填写：

In [23]:
frame.resample('D').ffill(limit=2)

,Colorado,Texas,New York,Ohio
2000-01-05,0.138355,1.881517,0.655367,1.496932
2000-01-06,0.138355,1.881517,0.655367,1.496932
2000-01-07,0.138355,1.881517,0.655367,1.496932
2000-01-08,NaN,NaN,NaN,NaN
2000-01-09,NaN,NaN,NaN,NaN
2000-01-10,NaN,NaN,NaN,NaN
2000-01-11,NaN,NaN,NaN,NaN
2000-01-12,-1.125212,-0.824337,0.803721,-0.672660


# 3 Resampling with Periods（对周期进行重采样）

对周期的索引进行重采样的过程，与之前时间戳的方法相似：

In [15]:
frame = pd.DataFrame(np.random.randn(24, 4),
                     index=pd.period_range('1-2000', '12-2001',
                                           freq='M'),
                     columns=['Colorado', 'Texas', 'New York', 'Ohio'])
frame[:5]

,Colorado,Texas,New York,Ohio
2000-01,-0.102209,0.510701,-1.114371,-0.894636
2000-02,0.862036,0.294313,-1.872954,-1.199034
2000-03,-0.222473,-0.387635,1.065752,-0.709758
2000-04,0.943072,-0.025638,1.384574,0.317012
2000-05,0.004031,0.408366,-0.528230,-1.367000


In [16]:
annual_frame = frame.resample('A-JAN').mean()
annual_frame

,Colorado,Texas,New York,Ohio
2000,-0.102209,0.510701,-1.114371,-0.894636
2001,0.063004,-0.290937,0.011009,-0.119317
2002,-0.582628,0.056482,-0.276555,-0.058362


增采样需要考虑的要多一些，比如在重采样前，选择哪一个时间跨度作为结束，就像asfreq方法那样。convertion参数默认是'start'，但也能用'end'：

In [28]:
# Q-DEC: Quarterly, year ending in December
annual_frame.resample('Q-DEC').ffill()

,Colorado,Texas,New York,Ohio
2000Q1,0.208662,-0.109971,-0.233464,0.138465
2000Q2,0.208662,-0.109971,-0.233464,0.138465
2000Q3,0.208662,-0.109971,-0.233464,0.138465
2000Q4,0.208662,-0.109971,-0.233464,0.138465
2001Q1,-0.401946,0.368050,-0.209196,-0.155851
2001Q2,-0.401946,0.368050,-0.209196,-0.155851
2001Q3,-0.401946,0.368050,-0.209196,-0.155851
2001Q4,-0.401946,0.368050,-0.209196,-0.155851
